In [ ]:
====================================================== 
###        Clean + Filter + Process occurrences       
###             By Cristina Ronquillo (2022)            
======================================================

In [ ]:
### Load packages
library(data.table)  # big dataset manipulation
library(tidyverse)  # data manipulation
library(plyr)  # data manipulation
library(dplyr)  # data manipulation
library(stringr)  # string manipulation

library(GADMTools)  # administrative units
library(raster)  # gis
library(rgdal)   # gis
library(sf)  # gis
library(sp)  # gis
library(seegSDM)  # gis
library(spatialEco)  # point in polygon tool

library(plantR)  # data cleaning functions
library(CoordinateCleaner)  # data cleaning functions

library(biogeo) # data cleaning duplicated data

library(lubridate) # dates
library(countrycode)  # country names standardization

### Read occurrences csv and check dimensions

In [ ]:
data <- data.table::fread("", encoding = 'UTF-8') 

dim(data)
head(data)
tail(data)

### Small version of the dataset with neccesary field

In [ ]:
colnames(data)
fields <- c("gbifID", "family", "genus", "species", "infraspecificEpithet", "taxonRank", "scientificName", "countryCode", "locality", "stateProvince", 
             "decimalLatitude",	"decimalLongitude", "coordinatePrecision", "day", "month", "year", "occurrenceStatus", "basisOfRecord", 
             "recordedBy", "issue")
data <- data[ , fields, with=FALSE]
colnames(data)
rm(fields)

### Apply Previous filters

In [ ]:
# Check and filter only presences (discard absences)
data <- data[occurrenceStatus == 'PRESENT', ]

In [ ]:
# Check Basis of Record types
unique(data$basisOfRecord)
# And how many records of each basisOfRecord:
data[, .N, by=basisOfRecord]

In [ ]:
# Remove records without appropriate basis of record:
data <- data[!basisOfRecord == 'FOSSIL_SPECIMEN', ]  # ! means exclude

## Taxonomic filters

In [ ]:
# Is the record identified at a proper taxonomic rank? Let's see what do we have in our dataset:
unique(data$taxonRank)
# And how many of each taxonRank:
data[, .N, by = taxonRank]

In [ ]:
# Filter those records with appropriate taxonomic rank
data <- data[taxonRank =='SUBSPECIES' | taxonRank == 'VARIETY'| taxonRank == 'FORM'| taxonRank =='SPECIES', ]

In [ ]:
# Create a checklist with scientific names
checklist <- data[ ,c('family', 'genus', 'species', 'infraspecificEpithet', 'taxonRank', 'scientificName') ]
checklist <- data.frame(unique(checklist)) # remove duplicated entries

## Geographical check

### 1. Check coordinates values:

In [ ]:
# Discard records with latitude/longitude values equals to zero or NULL and exact same value
data <- data[decimalLatitude!=0 & decimalLongitude!=0, ][decimalLatitude!=decimalLongitude, ]

### 2. Check coordinates precision (as number of decimal digits):

In [ ]:
#### Function to count number of decimals:
decimal.num.count <- function(x) {
    stopifnot(class(x) == "character")
    x <- ifelse(grepl("\\.", x), sub(".*\\.(.*)", "\\1", x), "")
    nchar(x)
}

In [ ]:
#### Apply function to latitude and longitude fields
latDigits <- decimal.num.count(as.character(data$decimalLatitude))
data <- cbind(data, latDigits)
lonDigits <- decimal.num.count(as.character(data$decimalLongitude))
data <- cbind(data, lonDigits)

In [ ]:
# Filter coordinates with 1 or more digit places
data <- data[latDigits > 0 & lonDigits > 0, ] 

### 3. Check coordinates position

#### A) Check if coordinates are placed in the assigned country through point in polygon analysis

In [ ]:
# Import gpkg of the world with Adm. Units borders at country level:
countries <- readOGR('gadm36_levels.gpkg', "level0") 

In [ ]:
# Transform occurrences into spatial points and project:
datapoints <- st_as_sf(x = data, coords = c("decimalLongitude", "decimalLatitude"), crs = "+proj=longlat +datum=WGS84 +no_defs")

In [ ]:
# Point in polygon execution:
pip <- point.in.poly(datapoints, countries)

In [ ]:
# Retrieve data as data table again
data <- pip@data
data$GID_0 <- NULL  # keep only 'Name_0' (country names of GADM)
setDT(data)  # set data table
setnames(data,"NAME_0", "GADM_Location")  # rename new field

In [ ]:
# Translate 'countryCode' information (ISO 3166-1-alpha-2 = "iso2c") into country names of GADM 'countryCode'
data$countryName <- countrycode(data$countryCode, origin = "iso2c", destination = "country.name")

In [ ]:
# Make changes in country names to match them
data$countryName[data$countryName == "Czechia"] <- "Czech Republic" # Change 'Czechia'='Czech Republic'
data$countryName[data$countryName == "Svalbard & Jan Mayen"] <- 'Svalbard and Jan Mayen'
data$countryName[data$countryName == "Eswatini"] <- "Swaziland"
data$countryName[data$countryName == "St. Pierre & Miquelon"] <- "Saint Pierre and Miquelon"
data$countryName[data$countryName == "Turks & Caicos Islands"] <- "Turks and Caicos Islands"
data$countryName[data$countryName == "Åland Islands"] <- "Finland"
data$countryName[data$countryName == "Palestinian Territories"] <- "Israel"
data$countryName[data$countryName == "Myanmar (Burma)"] <- "Myanmar"
data$countryName[data$countryName == "North Macedonia"] <- "Macedonia"
data$countryName[data$countryName == "Hong Kong SAR China"] <- "Hong Kong"
data$countryName[data$countryName == "Bosnia & Herzegovina"] <- "Bosnia and Herzegovina"

data$GADM_Location[data$GADM_Location == "Akrotiri and Dhekelia"] <- "Cyprus"
data$GADM_Location[data$GADM_Location == "Åland"] <- "Finland"
data$GADM_Location[data$GADM_Location == "Palestina"] <- "Israel"
data$GADM_Location[data$GADM_Location == "Northern Cyprus"] <- "Cyprus"

In [ ]:
# Match country names and label as 'FALSE' occurrences to check
data$countryCheck <- data$GADM_Location == data$countryName
data <- data %>% complete(data, fill = list(countryCheck = 'FALSE'))
setDT(data)

In [ ]:
# Subset and extract records located in country assigned by collector ('correct')
data_correct <- data[countryCheck == 'TRUE', ]

#### B) Check if coordinates are placed in correct habitat (sea-land)

In [ ]:
# Filter occurrences that didn't fall in the correct country polygon. Label those that didn't fall in any country polygon as 'SEA'
data_1 <- data[countryCheck == 'FALSE', ]
data_1 <- data_1 %>% complete(data_1, fill = list(GADM_Location = 'SEA'))
setDT(data_1)

In [ ]:
# Subset and check points that fall out of their habitat GADM_Location =='SEA'
data_sea_points <- data_1[GADM_Location == 'SEA', ]

In [ ]:
# Transform occurrences into spatial points and project:
data_sea_points <- st_as_sf(x = data_sea_points, coords = c("decimalLongitude", "decimalLatitude"), crs = "+proj=longlat +datum=WGS84 +no_defs")

##### Label points that fall in a fixed distance from the coastline 

In [ ]:
# Load buffer shapefile (here 0.1 degrees - ca. 10 km)
landBuff <- readOGR('buffer_0.1.shp')

In [ ]:
# Crop buffer shp with records extent to better performance:
new_extent <- as(extent(data_sea_points), 'SpatialPolygons') 
buff_0.1 <- crop(landBuff, new_extent)
proj4string(buff_0.1) <- "+proj=longlat +datum=WGS84 +no_defs"

In [ ]:
# Function to overlap points with buffer
sea_0.1 <- over(data_sea_points, buff_0.1)

In [ ]:
# Transform our initial dataset into a dataframe again
data_sea_points <- data_sea_points@data

In [ ]:
# Bind the column that indicates if the occurrence fall in our buffer (= Coastline). Occurrences out of the buffer have NA
data_sea_points <- cbind(data_sea_points, sea_0.1$featurecla) 
setDT(data_sea_points) # transform df to data table
setnames(data_sea_points, "featurecla", "shoreLine0.1")

In [ ]:
# Subset those points that fall in our buffer
dataSeaPointsGood <- data_sea_points[shoreLine0.1 == 'Coastline', ]

In [ ]:
##### NOW MOVE TO QGIS & Apply 'Join attributes by nearest' -> Assign the nearest country to each of these 'sea' occurrences 

In [ ]:
# Load output from QGIS
sea_buff_check <- fread('dataSeaPoints_coastlineCheck.csv',sep="\t")
setnames(sea_buff_check, "NAME_0", "Coastal_Country")
sea_buff_check$feature_x <- NULL
sea_buff_check$feature_y <- NULL
sea_buff_check$n <- NULL
unique(sea_buff_check$Coastal_Country)

In [ ]:
# Transform some country names:
sea_buff_check$Coastal_Country[sea_buff_check$Coastal_Country == "Åland"] <- "Finland"
sea_buff_check$Coastal_Country[sea_buff_check$Coastal_Country == "St. Pierre and Miquelon"] <- "St. Pierre & Miquelon"
sea_buff_check$Coastal_Country[sea_buff_check$Coastal_Country == "Turks and Caicos Islands"] <- "Turks & Caicos Islands"
sea_buff_check$Coastal_Country[sea_buff_check$Coastal_Country == "Isle of Man"] <- "United Kingdom"

In [ ]:
# Check if they are placed in the assigned country and subset those who are correctly located
sea_buff_check$country_shore_check <- sea_buff_check$Coastal_Country == sea_buff_check$countryName
setDT(sea_buff_check)
colnames(sea_buff_check)

In [ ]:
# Filter correct records:
data_sea_correct <- sea_buff_check[country_shore_check == 'TRUE', ]

In [ ]:
# Merge with previous correct dataset
data_correct <- rbind(data_correct, data_sea_correct, fill = TRUE)

##### Merge datasets with incorrect records (Check manually these records and add to data_correct dataset after corrections):

In [ ]:
# Subset points that fall in wrong country
incorrect_points <- data_1[!GADM_Location == 'SEA', ]  

In [ ]:
# Subset points that are placed in sea but outside 0.1 buffer from coast
incorrect_points2 <- data_sea_points[is.na(shoreLine0.1), ]  
incorrect_points2$shoreLine0.1 <- NULL

In [ ]:
# Subset points that are placed in coast buffer but nearest country is wrong
incorrect_points3 <- sea_buff_check[country_shore_check == 'FALSE', ]  
colnames(incorrect_points3)
incorrect_points3 <- incorrect_points3[ ,c(1:25, 27)]  # select some necessary fields

In [ ]:
# Merge everything
incorrect_points <- rbind(incorrect_points, incorrect_points2)
incorrect_points <- rbind(incorrect_points, incorrect_points3, fill = TRUE)

#### C) Check records that don't meet some location criteria 

In [ ]:
# Label coordinates placed in centroids of the country
cap <- cc_cap (data_correct,
              lon = "decimalLongitude",
              lat = "decimalLatitude",
              value = "flagged")
data_correct <- cbind(data_correct, cap)

In [ ]:
# Label coordinates placed in gbif headquarters
gbif <- cc_gbif(data_correct,
              lon = "decimalLongitude",
              lat = "decimalLatitude",
              value = "flagged")
data_correct <- cbind(data_correct, gbif)

In [ ]:
# Label coordinates from museums, gardens, institutions, zoo's... 
inst <- cc_inst(data_correct,
                lon = "decimalLongitude",
                lat = "decimalLatitude",
                value = "flagged")
data_correct <- cbind(data_correct, inst)

In [ ]:
# Filter and exclude centroids
data_correct <- data_correct[cap == 'TRUE', ][inst == 'TRUE', ][gbif == 'TRUE', ]